In [18]:
# Libraries
import requests
import json
import pandas as pd
import pprint
import os
import re
import config_manager as cnf
import github_api.github_api_manager as gapi


In [19]:

# 入力値
owner = "microsoft"
repo = "vscode"

print("入力値:{0} {1}".format(owner, repo))

入力値:microsoft vscode


In [20]:
# load config file
cnf_data = cnf.confg_manager()
cnf_data.load()
username, userpassword = cnf_data.get_userparam()

# create github api object
gapi_ctrl = gapi.github_api_manager(username, userpassword)

In [21]:
github_api = "/repos/{owner}/{repo}".replace("{owner}", owner).replace("{repo}", repo)
ret_json = gapi_ctrl.get_json(github_api)


GET https://api.github.com/repos/microsoft/vscode


In [24]:

check_list = ["name","full_name","description", "size", "stargazers_count", "watchers_count","forks_count","open_issues_count","network_count","subscribers_count"]

print("== とりあえず出力したい情報の一覧 ==")
for k in check_list:
    print(" {0} : {1}".format(k, str(ret_json[k])))


== とりあえず出力したい情報の一覧 ==
 name : vscode
 full_name : microsoft/vscode
 description : Visual Studio Code
 size : 296617
 stargazers_count : 101268
 watchers_count : 101268
 forks_count : 15962
 open_issues_count : 5438
 network_count : 15962
 subscribers_count : 2950


In [27]:
github_api = "/repos/{owner}/{repo}/languages".replace("{owner}", owner).replace("{repo}", repo)
ret_json = gapi_ctrl.get_json(github_api)

print("== 言語一覧 ==")
for k in ret_json:
    print(" {0} : {1}".format(k, str(ret_json[k])))

GET https://api.github.com/repos/microsoft/vscode/languages
== 言語一覧 ==
 TypeScript : 30071907
 JavaScript : 1089542
 CSS : 474678
 Inno Setup : 250395
 HTML : 175131
 Shell : 33922
 Batchfile : 8943
 PowerShell : 6190
 Dockerfile : 4631
 Groovy : 3928
 C++ : 3251
 Python : 2405
 Makefile : 2162
 Ruby : 1703
 Objective-C++ : 1387
 Objective-C : 1387
 Clojure : 1206
 Raku : 1065
 PHP : 998
 Visual Basic .NET : 893
 Perl : 857
 C : 818
 Go : 652
 F# : 634
 Java : 599
 CoffeeScript : 590
 Rust : 532
 C# : 488
 R : 362
 Roff : 351
 ShaderLab : 330
 Swift : 284
 Lua : 252
 HLSL : 184
 Hack : 16


In [ ]:
github_api = "/repos/{owner}/{repo}/contributors".replace("{owner}", owner).replace("{repo}", repo)
ret_json = gapi_ctrl.get_json(github_api)

print("== 言語一覧 ==")
for k in ret_json:
    print(" {0} : {1}".format(k, str(ret_json[k])))